In [5]:
import numpy as np
import pandas as pd
from sklearn import model_selection, preprocessing
import xgboost as xgb

import datetime
from scipy.stats import norm

print('Running Model 2...')
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
id_train = train.id
id_test = test.id

mult = .969

y_train = train["price_doc"] * mult + 10
x_train = train.drop(["id", "timestamp", "price_doc"], axis=1)
x_test = test.drop(["id", "timestamp"], axis=1)

for c in x_train.columns:
    if x_train[c].dtype == 'object':
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(x_train[c].values))
        x_train[c] = lbl.transform(list(x_train[c].values))

for c in x_test.columns:
    if x_test[c].dtype == 'object':
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(x_test[c].values))
        x_test[c] = lbl.transform(list(x_test[c].values))

xgb_params = {
    'eta': 0.05,
    'max_depth': 5,
    'subsample': 0.7,
    'colsample_bytree': 0.7,
    'objective': 'reg:linear',
    'eval_metric': 'rmse',
    'silent': 1
}

dtrain = xgb.DMatrix(x_train, y_train)
dtest = xgb.DMatrix(x_test)

num_boost_rounds = 385  # This was the CV output, as earlier version shows
model_2 = xgb.train(dict(xgb_params, silent=0), dtrain, num_boost_round= num_boost_rounds)

test_predict_2 = model_2.predict(dtest)
train_predict_2 = model_2.predict(dtrain)
test_predict_df_2 = pd.DataFrame({'id': id_test, 'price_doc': test_predict_2})
train_predict_df_2 = pd.DataFrame({'id': id_train, 'price_doc': train_predict_2})

Running Model 2...


In [9]:
train_predict_df_2.to_csv("train_pred_df_2.csv")
test_predict_df_2.to_csv("test_pred_df_2.csv")